#model training

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/krishnaik06/FSDSRegression/main/notebooks/data/gemstone.csv')


In [3]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df = df.drop(labels='id',axis = 1)

In [5]:
# independent and dependent variables

X = df.drop(labels = 'price',axis = 1 )
Y = df[['price']]

In [7]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [8]:
# define which columns should be ordinal encoded and which should be scaled
categorical_columns = X.select_dtypes(include='object').columns
numerical_columns = X.select_dtypes(exclude='object').columns

In [9]:
cut_categories = ['Fair','Good','Very Good','Premium','Ideal']
color_categories = ['D','E','F','G','H','I','J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [10]:
Y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [15]:
from sklearn.impute import SimpleImputer #handling missing missing values
from sklearn.preprocessing import StandardScaler # handling feature scaling
from sklearn.preprocessing import OrdinalEncoder #ordinal encoding
##pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [17]:
# Numerical Pipeline

num_pipeline = Pipeline(

    steps = [
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

cat_pipeline = Pipeline(

    steps = [
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinal_encoder', OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor = ColumnTransformer([

    ('num_pipeline',num_pipeline,numerical_columns),
    ('cat_pipeline',cat_pipeline,categorical_columns)
])

In [19]:
#Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.30,random_state = 36)

In [21]:
preprocessor.fit_transform(X_train)


array([[ 0.23744474, -0.94489036,  1.9602201 , ..., -1.13977731,
         1.53220929, -0.64864203],
       [ 0.23744474,  0.16480812,  1.44038437, ..., -0.13307011,
         0.30060839, -0.64864203],
       [-1.05915851,  0.44223274, -1.67863004, ...,  0.87363709,
         0.30060839,  2.676179  ],
       ...,
       [ 0.47515533, -0.29756625, -1.15879431, ...,  0.87363709,
        -0.31519207, -1.31360624],
       [ 0.23744474,  0.71965736,  2.48005584, ..., -0.13307011,
        -1.54679298, -0.64864203],
       [-0.60534737, -0.20509138,  0.4007129 , ..., -1.13977731,
         0.30060839,  0.01632217]])

In [23]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())

In [24]:
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [25]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.237445,-0.944890,1.960220,0.519313,0.472919,0.386137,-1.139777,1.532209,-0.648642
1,0.237445,0.164808,1.440384,0.375111,0.336869,0.371649,-0.133070,0.300608,-0.648642
2,-1.059159,0.442233,-1.678630,-1.256173,-1.286667,-1.221975,0.873637,0.300608,2.676179
3,-0.173146,0.904607,0.920549,-0.030457,-0.007792,0.067412,-1.139777,-0.930993,-1.313606
4,0.453545,-2.239539,1.440384,0.672527,0.754090,0.458574,-1.139777,0.300608,2.011215


In [26]:
##Model Training

from sklearn.linear_model import LinearRegression, Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [27]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [28]:
regression.coef_

array([[ 6428.0600595 ,  -129.96363815,   -66.5201496 , -2054.43596194,
         -120.22065555,   -99.38786289,    72.22779539,  -463.65541534,
          652.08151637]])

In [29]:
regression.intercept_

array([3964.91854673])